In [ ]:
# # Setting up google drive 
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)
# import sys
# sys.path.append('/content/gdrive/MyDrive/Colab Notebooks')

In [ ]:
# %pip install torch

In [ ]:
import my_utils as mu
import torch
from torch import nn

# The Task

* Our **Task** for this week is to implement LeNet.
* The Learning Outcome: Hands-on application of PyTorch's API for creating and training CNNs.



# LeNet

* At a high level, LeNet (LeNet-5) consists of 2 parts:
    1. a convolutional encoder consisting of two convolutional layers; and
    2. a dense block consisting of three fully-connected layers;
<!-- 
![Data flow in LeNet. The input is a handwritten digit, the output a probability over 10 possible outcomes.](img/lenet.svg)  -->

<!-- ![Data flow in LeNet. The input is a handwritten digit, the output a probability over 10 possible outcomes.](https://drive.google.com/uc?export=view&id=18Kd-JNGeKp38qAVEuxEyYU7rjNudWdWA) -->   




# LeNet -- Convolutional Encoder

* Each convolutional *block* consists of: 
    * A convolutional layer.
    * A sigmoid activation function (ReLUs were discovered recently).
    * A subsequent average pooling operation (max pooling was discovered later).
* Each convolutional layer uses a $5\times 5$ kernel.
* The first convolutional layer has 6 output channels, while the second has 16.
* Each $2\times2$ pooling operation (stride 2) reduces dimensionality by a factor of $4$ via spatial downsampling.
* The convolutional block emits an output with shape given by (batch size, number of channels, height, width).



# LeNet -- Dense Block

* In order to pass the output from the convolutional block to the dense block, we must flatten each example in the minibatch.
* In other words, we take the four-dimensional input and transform it into the two-dimensional input expected by fully-connected layers:
    * the two-dimensional representation that we desire uses the first dimension to index examples in the minibatch
    * the second to give the flat vector representation of each example.
* LeNet's dense block has three fully-connected layers, with 120, 84, and 10 outputs, respectively.
    * Because we are still performing classification, the 10-dimensional output layer corresponds to the number of possible output classes.

# Compressed LeNet Representation 


<!-- ![Compressed notation for LeNet-5.](img/lenet-vert.svg) -->

 ![Compressed notation for LeNet-5.](https://drive.google.com/uc?export=view&id=1sL0P0_DwSU8vSOvcoV6GKkwwokyuKysS) 

# Concise Implementation of LeNet

* Goal: use high-level APIs of PyTorch for implementing LeNet for classification. 

In [ ]:
# Read training and test data
batch_size = 256
train_iter, test_iter = mu.load_data_fashion_mnist(batch_size)
type(train_iter)

# Defining the Model

* We will modify the code from MLP




In [ ]:
class LeNet(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LeNet, self).__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.conv1 = nn.Conv2d(in_channels=num_inputs, out_channels=6, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        self.p1 = nn.AvgPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16, kernel_size=5)
        self.p2 = nn.AvgPool2d(kernel_size=2,stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=400,out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84,out_features=10)
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.p1(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.p2(out)
        out = self.flatten(out)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d: # by checking thr type we can init different layers in different ways
        torch.nn.init.xavier_uniform_(m.weight)          

num_outputs = 10
model = LeNet(num_inputs=1, num_outputs=10)

model.apply(init_weights);
print(model)

# Loss and Optimization Algorithm
* As in Softmax Regression

In [ ]:
loss = nn.CrossEntropyLoss()
lr = 0.9
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# Training

* Use `my_utils.train_ch3` as in Softmax Regression

In [ ]:
num_epochs = 20
mu.train_ch3(model, train_iter, test_iter, loss, num_epochs, optimizer)